In [19]:
import asyncio
import json
import logging
import os
from datetime import datetime, timezone
from logging import INFO

from dotenv import load_dotenv

from graphiti_core import Graphiti
from graphiti_core.nodes import EpisodeType
from graphiti_core.search.search_config_recipes import NODE_HYBRID_SEARCH_RRF


# Configure logging
logging.basicConfig(
    level=INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)
logger = logging.getLogger(__name__)


load_dotenv()

# Neo4j connection parameters
# Make sure Neo4j Desktop is running with a local DBMS started
neo4j_uri = os.environ.get('NEO4J_URI', 'bolt://localhost:7687')
neo4j_user = os.environ.get('NEO4J_USER', 'neo4j')
neo4j_password = os.environ.get('NEO4J_PASSWORD', 'password')


class ChatMemory:
    def __init__(self, uri, user, password):
        self.driver = AsyncGraphDatabase.driver(uri, auth=(user, password))
        self.graphiti = Graphiti(uri=uri, user=user, password=password)

    async def save_message_as_episode(self, user_id: str, text: str, sender: str):
        timestamp = datetime.now(timezone.utc).isoformat()

        await self.graphiti.add_episode(
            external_user_id=user_id,
            content=text,
            sender=sender,
            type=EpisodeType.chat,
            timestamp=timestamp,
        )


async def run_conversation():
    memory = ChatMemory(neo4j_uri, neo4j_user, neo4j_password)

    user_id = "user-123"
    user_input = "I need help with my order"

    # Step 1: Save user message to memory
    await memory.save_message_as_episode(user_id, user_input, sender="user")

    # Step 2: Gemini reply
    response = gemini_model.generate_content(user_input)
    reply = response.text.strip()

    # Step 3: Save Gemini response to memory
    await memory.save_message_as_episode(user_id, reply, sender="agent")

    print("Gemini Reply:", reply)


    await run_conversation()



ValueError: NEO4J_URI, NEO4J_USER, and NEO4J_PASSWORD must be set

In [28]:
import os

print("Current working directory:", os.getcwd())
print("Files in this directory:", os.listdir())

Current working directory: /Users/mahmoud/agent-with-graphiti/notebooks
Files in this directory: ['setup_neo4j.ipynb']
